In [1]:
def counter():
    i = 0
    def inc():
        nonlocal i
        i  += 1
        return i
    return inc


In [2]:
c = counter()
c(), c(), c(), c()

(1, 2, 3, 4)

In [3]:
help(iter)  # not very helpfull... but it's possible to pass sentinel value (in a second form)

Help on built-in function iter in module builtins:

iter(...)
    Get an iterator from an object.

    In the first form, the argument must supply its own iterator, or be a sequence.
    In the second form, the callable is called until it returns the sentinel.



In [4]:
class CounterInterator:
    def __init__(self, counter_callable):
        self.counter_callable = counter_callable

    def __iter__(self):
        return self

    def __next__(self):
        return self.counter_callable()


In [5]:
counter_ = counter()
counter_iterator = CounterInterator(counter_)

In [6]:
for _ in range(5):
    print(next(counter_iterator))

1
2
3
4
5


In [7]:
class CounterInterator:
    def __init__(self, counter_callable, sentinel):
        self.counter_callable = counter_callable
        self.sentinel = sentinel

    def __iter__(self):
        return self

    def __next__(self):
        r = self.counter_callable()
        if r == self.sentinel:
            raise StopIteration
        return r

In [8]:
counter_ = counter()
counter_iterator = CounterInterator(counter_, 12)

In [9]:
list(counter_iterator)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [10]:
next(counter_iterator), next(counter_iterator)  # still possible to iterate

(13, 14)

In [11]:
class CounterInterator:
    def __init__(self, counter_callable, sentinel):
        self.counter_callable = counter_callable
        self.sentinel = sentinel
        self.is_consumed = False

    def __iter__(self):
        return self

    def __next__(self):
        if self.is_consumed:
            raise StopIteration

        r = self.counter_callable()
        if r == self.sentinel:
            self.is_consumed = True
            raise StopIteration
        return r


In [12]:
counter_ = counter()
counter_iterator = CounterInterator(counter_, 12)
list(counter_iterator)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [13]:
try:
    print(next(counter_iterator))
    print(next(counter_iterator))
    print(next(counter_iterator))
except StopIteration as e:
    print(e)

In [14]:
class CallableInterator:
    def __init__(self, callable_, sentinel):
        self.callable_ = callable_
        self.sentinel = sentinel
        self.is_consumed = False

    def __iter__(self):
        return self

    def __next__(self):
        if self.is_consumed:
            raise StopIteration

        r = self.callable_()
        if r == self.sentinel:
            self.is_consumed = True
            raise StopIteration
        return r

In [15]:
counter_ = counter()
counter_iterator = CallableInterator(counter_, 15)
list(counter_iterator)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [16]:
counter_ = counter()
counter_iter = iter(counter_, 5)
list(counter_iter)

[1, 2, 3, 4]

In [17]:
def concat_a(s: str = ""):
    def run():
        nonlocal s
        s += "a"
        return s

    return run

func_ = concat_a("aa")
counter_iter = iter(func_, "a" * 10)
list(counter_iter)

['aaa', 'aaaa', 'aaaaa', 'aaaaaa', 'aaaaaaa', 'aaaaaaaa', 'aaaaaaaaa']

### Delegating iterators

In [18]:
from collections import namedtuple


Person = namedtuple("Person", "first last")


class PersonNames:
    def __init__(self, persons: list[Person]):
        try:
            self._persons =  [
                person.first.capitalize() 
                + " " + person.last.capitalize()
                for person  in persons
            ]
        except (TypeError, AttributeError) as e:
            print(e)
            self._persons = []
        
        

In [19]:
persons = [
    Person("mikeL", "palIn"),
    Person("Bob", "Man"),
    Person("spidEr", "man"),
    Person("bAt", "mAN"),
]

In [20]:
persons_names = PersonNames(persons)

In [21]:
persons_names._persons

['Mikel Palin', 'Bob Man', 'Spider Man', 'Bat Man']

In [22]:
from collections import namedtuple


Person = namedtuple("Person", "first last")


class PersonNames:
    def __init__(self, persons: list[Person]):
        try:
            self._persons =  [
                person.first.capitalize() 
                + " " + person.last.capitalize()
                for person  in persons
            ]
        except (TypeError, AttributeError) as e:
            print(e)
            self._persons = []
        

    def __iter__(self):
        return iter(self._persons)   # we can delegate creation of iterator

In [23]:
persons = [
    Person("mikeL", "palIn"),
    Person("Bob", "Man"),
    Person("spidEr", "man"),
    Person("bAt", "mAN"),
]
persons_names = PersonNames(persons)
list(persons_names)

['Mikel Palin', 'Bob Man', 'Spider Man', 'Bat Man']